# Environment Explore

In [84]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media
import jax.numpy as jp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_21_1_250_clip_0.p",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

# Creating env objects

In [86]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [87]:
key = random.PRNGKey(0)
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)

In [88]:
type(env._ref_traj)

mocap_preprocess.ReferenceClip

In [89]:
n = env.reset(key)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

shape of each of the reference trajectory

In [90]:
jax.tree_util.tree_map(
    lambda x: print(x.shape),
    env._ref_traj,
)

(250, 3)
(250, 5, 3)
(250, 18, 3)
(250, 18, 4)
(250, 3)
(250, 4, 3)
(250, 67)
(250, 67)
(0,)
(250, 3)
(250, 4)
(0,)
(250, 3)


ReferenceClip(angular_velocity=None, appendages=None, body_positions=None, body_quaternions=None, center_of_mass=None, end_effectors=None, joints=None, joints_velocity=None, markers=None, position=None, quaternion=None, scaling=None, velocity=None)

In [91]:
stacked = jp.hstack([env._ref_traj.position,
           env._ref_traj.quaternion,
           env._ref_traj.joints,
           ])

stacked[:,env.joint_order].shape

(250, 30)

In [92]:
len(env.joint_order)

30

# Creating state objects & reset_fn, get_obs_fn
`reset_fn` and `get_obs_fn` is called when initilizing the environment and creating the state object

In [93]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

In [94]:
state.pipeline_state.xpos[:,0] # all x values
state.pipeline_state.xpos[env.body_idxs].shape

(18, 3)

In [95]:
env._ref_traj.body_positions.shape

(250, 18, 3)

reference function check

In [96]:
env._ref_traj.joints_velocity[0,:] - state.pipeline_state.qvel[6:]

Array([-2.2884121e+00, -9.3691826e-02, -2.9669708e-01, -3.4307184e+00,
        1.9533709e-01, -4.7531295e+00, -6.1745608e-01, -9.0986192e-01,
        1.0734018e+01,  2.9669285e+00,  2.6955688e+00,  0.0000000e+00,
        4.8113151e+00, -1.2214106e+00,  7.3403511e+00, -3.1193109e+00,
       -7.4505806e-07,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  2.6349113e+00,  1.0424142e+01,
        6.8323698e+00, -1.5623410e+01, -1.1410461e+01, -9.6153784e+00,
        6.7606001e+00,  0.0000000e+00,  0.0000000e+00,  8.9186507e-01,
       -3.9957674e+00,  0.0000000e+00, -4.1885204e+00,  0.0000000e+00,
      

end effector position

In [97]:
state.pipeline_state.xpos[jp.array(env._end_eff_idx)]

Array([[-0.00912952,  0.4411053 ,  0.01141115],
       [-0.03021844,  0.414083  ,  0.01621009],
       [ 0.0780039 ,  0.40200663,  0.01433735],
       [ 0.08267315,  0.36600125,  0.02249428]], dtype=float32)

In [98]:
state.obs

Array([ 0.04518413,  0.40962166,  0.07110023, ..., -0.30106932,
        0.00287952,  0.03926991], dtype=float32)

In [99]:
jp.isclose(state.obs, 0)

Array([False, False, False, ..., False, False, False], dtype=bool)

In [100]:
jp.isclose(state.obs, 0).sum()/state.obs.shape[0]

Array(0.17094861, dtype=float32)

# Stepping Env: Test step_fn, reward_fn, & termination_fn
`step_fn`, which calls `reward_fn` & `termination_fn` is only used when actually making action, not when setting up environment and state object

In [101]:
# next = jit_reset(key)
# print("starting rollout")
# mu = 0
# sigma = .1
# rollout = []
# for _ in range(500):
#     next = jit_step(next, mu + sigma * random.normal(key, shape=(env.sys.nu,)))
#     rollout.append(next)

In [102]:
# import mediapy as media
# import os

# os.environ["MUJOCO_GL"] = "glfw"
# rollout_data = [s.pipeline_state for s in rollout]

# video = env.render(rollout_data, camera='side_alt', height=500, width=500)

# media.show_video(video, fps=1.0 / env.dt)

# Stepping Environment & Observe Changes in Frames

In [103]:
env = RodentSingleClipTrack(params)
reset_fn = jax.jit(jax.vmap(env.reset))
step_fn = jax.jit(jax.vmap(env.step))

n_envs = 1
key_envs = jax.random.split(key, n_envs)
env_state = reset_fn(key_envs)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitatio

In [104]:
env_state.metrics

{'healthy_time': Array([0.], dtype=float32),
 'ract': Array([0.], dtype=float32),
 'rapp': Array([0.], dtype=float32),
 'rcom': Array([0.], dtype=float32),
 'reset_num': Array([1.], dtype=float32),
 'rquat': Array([0.], dtype=float32),
 'rvel': Array([0.], dtype=float32),
 'termination_error': Array([0.], dtype=float32)}

In [105]:
mu = 0
sigma = 0.3
action =  mu + sigma * random.normal(key, shape=(n_envs, env.sys.nu))
new_state = step_fn(env_state, action)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/.brax-imitation/lib/python3.10/site-packages/mujoco

In [106]:
new_state.metrics

{'healthy_time': Array([1.], dtype=float32),
 'ract': Array([-0.00112245], dtype=float32),
 'rapp': Array([0.], dtype=float32),
 'rcom': Array([1.], dtype=float32),
 'reset_num': Array([1.], dtype=float32),
 'rquat': Array([1.], dtype=float32),
 'rvel': Array([1.], dtype=float32),
 'termination_error': Array([0.13742143], dtype=float32)}

In [107]:
new_state = step_fn(new_state, action)
new_state.metrics

{'healthy_time': Array([2.], dtype=float32),
 'ract': Array([-0.00112245], dtype=float32),
 'rapp': Array([0.], dtype=float32),
 'rcom': Array([0.99997914], dtype=float32),
 'reset_num': Array([1.], dtype=float32),
 'rquat': Array([0.04333283], dtype=float32),
 'rvel': Array([0.], dtype=float32),
 'termination_error': Array([0.41220427], dtype=float32)}

In [108]:
new_state.info

{'cur_frame': Array([41], dtype=int32),
 'reset_times': Array([1], dtype=int32, weak_type=True),
 'step_after_reset': Array([2], dtype=int32, weak_type=True),
 'termination_error_vnl': Array([0.41220427], dtype=float32)}

In [109]:
new_state = step_fn(new_state, action)
new_state.metrics

{'healthy_time': Array([3.], dtype=float32),
 'ract': Array([-0.00112245], dtype=float32),
 'rapp': Array([0.], dtype=float32),
 'rcom': Array([0.9997602], dtype=float32),
 'reset_num': Array([1.], dtype=float32),
 'rquat': Array([0.00081137], dtype=float32),
 'rvel': Array([0.], dtype=float32),
 'termination_error': Array([0.5782251], dtype=float32)}

In [110]:
# new_state = env_state
# check = []
# error = []
# for _ in range(1):
#     mu = 0
#     sigma = 0.3
#     action =  mu + sigma * random.normal(key, shape=(n_envs, env.sys.nu))
#     new_state = step_fn(new_state, action)
#     check.append(new_state.info['step_after_reset'])
#     error.append(new_state.info['termination_error'])

# print(check, error)

In [111]:
# new_state = step_fn(new_state, action)
# check_state = reset_fn(key_envs)
# check_state = step_fn(check_state, action)
# check_state.info['episode_frame'] # does reset correctly

# Test Wrapper For Environment

In [112]:
from brax import envs
wrap_for_training = envs.training.wrap

wrap_env = wrap_for_training(
    env,
    episode_length=1000,
    action_repeat=1
  )

In [113]:
wrap_env.env.env.env

# Comparing Relative Functions
Instantiate `dm_contol` MultiClipMocapTracking Environment with ur ref_traj data, seems a bit challenging, using rodent trajectory is not applicable probably (observation tell).

In [114]:
env_state.obs

Array([[-0.00060661,  0.4388396 ,  0.07299647, ..., -0.21182789,
        -0.03065387,  0.03926991]], dtype=float32)

In [115]:
from walker import Rat
from dm_control.locomotion.walkers import rescale

walker = Rat()
rescale.rescale_subtree(walker.mjcf_model, 0.9, 0.9)

In [116]:
# from dm_control import composer
# from dm_control.locomotion import arenas
# from dm_control.locomotion import walkers
# from dm_control.locomotion.mocap import props
# from dm_control.locomotion.tasks.reference_pose import tracking
# from dm_control.locomotion.tasks.reference_pose import types
# from dm_control.utils import io as resources
# import os
# import h5py

# # Define the path to your motion capture data file
# file_name = 'all_snips_250.h5'
# current_directory = os.getcwd()
# TEST_FILE_PATH = os.path.join(current_directory, file_name)

# with h5py.File(TEST_FILE_PATH, 'r') as f:
#     dataset_keys = tuple(f.keys())
#     dataset = types.ClipCollection(ids=dataset_keys,)

# # Set up the mocap tracking task
# task = tracking.MultiClipMocapTracking(
#     walker=walker, #walkers.CMUHumanoidPositionControlled,
#     arena=arenas.Floor(),
#     ref_path=resources.GetResourceFilename(TEST_FILE_PATH),
#     ref_steps=(1, 2, 3, 4, 5),
#     min_steps=1,
#     dataset=dataset,
#     reward_type='comic',
# )

# # Initialize the environment
# env = composer.Environment(task=task)
# env.reset()